## Calculo de tiempo promedio de horas pico de las sucursales 
### Desarrollado por:
- **Nombre:** Gabriel Chavez 
- **Fecha:** 2024-11-21

### Descripción
Tomaremos de referencia las transacciones de kielsa Honduras, donde pondremos de forma ordenada las cajas y los id de facturas, para poder calcular el tiempo promedio de horas pico de las sucursales.
- La hora de inicio sera la fecha de finazalizacion anterior.
- El tiempo promedio sera la diferencia entre la hora de inicio y la hora de fin de la transaccion.
- Organizar por tipo de transaccion y sacar el promedio.


In [2]:
import polars as pl
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import text

from sqlalchemy import create_engine
from urllib.parse import quote_plus


connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/BI_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)
engine = create_engine(connection_string)
query = text("""SELECT
    fc.Suc_Id,
    fc.Emp_Id,
    fc.Caja_Id,
    fc.Factura_Id,
    fc.Factura_Fecha,
    fc.Factura_FechaHora,
    fc.AnioMes_Id,
    fc.Cliente_Id,
    fc.MonederoTarj_Id,
    fc.Vendedor_Id,
    CASE 
        WHEN fc.Monedero_Id = '0' THEN 'No Identificada'
        ELSE md.Tipo_Plan 
    END AS Tipo_Plan,
    md.Edad,
    suc.TipoSucursal_Id,
    suc.TipoSucursal_Nombre,
    suc.Zona_Id,
    suc.Zona_Nombre
FROM
    [BI_Kielsa_Hecho_FacturaEncabezado] AS fc
    JOIN [BI_Kielsa_Dim_Monedero] AS md 
        ON md.Monedero_Id = fc.Monedero_Id AND md.Emp_Id = fc.Emp_Id       
    JOIN [BI_Kielsa_Dim_Sucursal] AS suc 
        ON suc.Sucursal_Id = fc.Suc_Id AND suc.Emp_Id = fc.Emp_Id
WHERE
    fc.AnioMes_Id = 202410 
    AND fc.Emp_Id = 1;""")

with engine.connect() as conn:
    df_base = pd.read_sql(query, conn, index_col='Factura_Id')


In [3]:
df_base.head()

,Suc_Id,Emp_Id,Caja_Id,Factura_Fecha,Factura_FechaHora,AnioMes_Id,Cliente_Id,MonederoTarj_Id,Vendedor_Id,Tipo_Plan,Edad,TipoSucursal_Id,TipoSucursal_Nombre,Zona_Id,Zona_Nombre
Factura_Id,,,,,,,,,,,,,,,
15073,118,1,3,2024-10-26,13:20:38,202410,10000001,0502-1996-01137,10002837,Monedero todo publico,28,8,Supermercado,2,NORTE
119584,56,1,1,2024-10-01,13:11:58,202410,10000001,0502-1996-01177,10003784,Monedero todo publico,28,3,Clínica/Hospitalaria,2,NORTE
17786,268,1,1,2024-10-22,16:26:31,202410,10000001,0502-1996-01178,10004530,Monedero todo publico,28,9,Plaza Comercial,2,NORTE
73545,248,1,4,2024-10-22,17:19:22,202410,10000001,0502-1996-01288,10004684,Monedero todo publico,29,9,Plaza Comercial,2,NORTE
22382,268,1,2,2024-10-30,19:09:34,202410,10000001,0502-1996-01632,10004530,Monedero todo publico,28,9,Plaza Comercial,2,NORTE


In [4]:
# copia de dataframe base
df = df_base.copy()

# Convertir los valores de hora a cadenas
df['Factura_FechaHora'] = df['Factura_FechaHora'].astype(str)

# Agregar una fecha ficticia a los valores de hora
df['Factura_FechaHora'] = '1900-01-01 ' + df['Factura_FechaHora']

# Convertir la columna Factura_FechaHora a datetime
df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Asegúrate de que Factura_Fecha y Factura_FechaHora sean tipo datetime
df['Factura_Fecha'] = pd.to_datetime(df['Factura_Fecha'])

df_base.head()

,Suc_Id,Emp_Id,Caja_Id,Factura_Fecha,Factura_FechaHora,AnioMes_Id,Cliente_Id,MonederoTarj_Id,Vendedor_Id,Tipo_Plan,Edad,TipoSucursal_Id,TipoSucursal_Nombre,Zona_Id,Zona_Nombre
Factura_Id,,,,,,,,,,,,,,,
15073,118,1,3,2024-10-26,13:20:38,202410,10000001,0502-1996-01137,10002837,Monedero todo publico,28,8,Supermercado,2,NORTE
119584,56,1,1,2024-10-01,13:11:58,202410,10000001,0502-1996-01177,10003784,Monedero todo publico,28,3,Clínica/Hospitalaria,2,NORTE
17786,268,1,1,2024-10-22,16:26:31,202410,10000001,0502-1996-01178,10004530,Monedero todo publico,28,9,Plaza Comercial,2,NORTE
73545,248,1,4,2024-10-22,17:19:22,202410,10000001,0502-1996-01288,10004684,Monedero todo publico,29,9,Plaza Comercial,2,NORTE
22382,268,1,2,2024-10-30,19:09:34,202410,10000001,0502-1996-01632,10004530,Monedero todo publico,28,9,Plaza Comercial,2,NORTE


In [33]:
import numpy as np

# Copia de DataFrame base
df = df_base.copy()

# Convertir Factura_FechaHora a cadena para agregar fecha ficticia
df['Factura_FechaHora'] = df['Factura_FechaHora'].astype(str)
df['Factura_FechaHora'] = '1900-01-01 ' + df['Factura_FechaHora']

# Convertir Factura_FechaHora a tipo datetime
df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], errors='coerce')

# Asegurarse de que Factura_Fecha sea tipo datetime
df['Factura_Fecha'] = pd.to_datetime(df['Factura_Fecha'])

# Ordenar el DataFrame
df = df.sort_values(by=['Suc_Id', 'Caja_Id', 'Factura_Id', 'Factura_Fecha'], ascending=[True, True, True, True])

# Crear columna Factura_Inicio_FechaHora solo si la fecha es la misma
df['Factura_Inicio_FechaHora'] = np.where(
    df['Factura_Fecha'] == df['Factura_Fecha'].shift(1),
    df['Factura_FechaHora'].shift(1),
    pd.NaT  # Asignar NaT si no coinciden
)

# Asegurarse de que la columna Factura_Inicio_FechaHora sea tipo datetime
df['Factura_Inicio_FechaHora'] = pd.to_datetime(df['Factura_Inicio_FechaHora'], errors='coerce')

# Calcular el tiempo transcurrido
df['Tiempo_Transcurrido'] = df['Factura_FechaHora'] - df['Factura_Inicio_FechaHora']

# convertir el tiempo transcurrido a minutos
df['Tiempo_Transcurrido'] = df['Tiempo_Transcurrido'].dt.total_seconds() / 60

# Revisar resultados
print(df[['Factura_Fecha', 'Factura_FechaHora', 'Factura_Inicio_FechaHora', 'Tiempo_Transcurrido']].head())


           Factura_Fecha   Factura_FechaHora Factura_Inicio_FechaHora  \
Factura_Id                                                              
26191         2024-10-01 1900-01-01 13:36:13                      NaT   
26192         2024-10-01 1900-01-01 14:31:49      1900-01-01 13:36:13   
26193         2024-10-01 1900-01-01 15:19:52      1900-01-01 14:31:49   
26194         2024-10-01 1900-01-01 15:27:05      1900-01-01 15:19:52   
26195         2024-10-01 1900-01-01 19:03:44      1900-01-01 15:27:05   

            Tiempo_Transcurrido  
Factura_Id                       
26191                       NaN  
26192                 55.600000  
26193                 48.050000  
26194                  7.216667  
26195                216.650000  


In [35]:
# Crear columna de diferencia de tiempo entre la Factura_Inicio_FechaHora y Factura_FechaHora en el caso que Factura_Inicio_FechaHora no sea NaT
# Filtrar el DataFrame por la fecha '2024-10-01' en la columna 'Factura_Fecha'
filtered_df = df[df['Factura_Fecha'] == '2024-10-03']

# Mostrar las primeras filas del DataFrame filtrado
print(filtered_df.head())

# guardar en un excel
filtered_df.to_excel('facturas_2024-10-03.xlsx', index=False)

            Suc_Id  Emp_Id  Caja_Id Factura_Fecha   Factura_FechaHora  \
Factura_Id                                                              
26206            1       1        1    2024-10-03 1900-01-01 07:42:40   
26207            1       1        1    2024-10-03 1900-01-01 07:43:09   
26208            1       1        1    2024-10-03 1900-01-01 07:43:42   
26209            1       1        1    2024-10-03 1900-01-01 07:45:17   
26210            1       1        1    2024-10-03 1900-01-01 07:45:51   

            AnioMes_Id  Cliente_Id  MonederoTarj_Id  Vendedor_Id  \
Factura_Id                                                         
26206           202410    10000001  1801-2004-02053     10004092   
26207           202410    10000002  1801-1984-01948     10004092   
26208           202410    10000001  1801-1966-00159     10004092   
26209           202410    10000002  3801-1984-00094     10004092   
26210           202410    10000002  1801-1953-00331     10004092   

           